# Aggregration of Experiment Results

This notebook pulls results from the `output` folder and aggregates them in a large table that can than be formated, pivoted etc.

In [6]:
from pathlib import Path

OUTPUT_PATH = Path("/work/smt4/thulke/vieweg/SER/Code/sisyphus/output")

## Metrics

In [7]:
import pandas as pd
import glob

dfs = []
for path in map(Path, glob.glob(f"{OUTPUT_PATH}/**/metrics.csv", recursive=True)):
    if "pp" in path.parts:
        continue

    metrics_df = pd.read_csv(path)
    metrics_df["experiment"] = path.parts[-3]
    metrics_df["dataset"] = path.parts[-4]
    dfs.append(metrics_df)

metrics = pd.concat(dfs, axis=0, ignore_index=True)
metrics.head()

,split,metric,value,experiment,dataset
0,train,eer,0.059028,lj_finetune_facebook_wav2vec2-base-960h,rav
1,val,eer,0.312500,lj_finetune_facebook_wav2vec2-base-960h,rav
2,test,eer,0.345833,lj_finetune_facebook_wav2vec2-base-960h,rav
3,train,beer_Happiness,0.215278,lj_finetune_facebook_wav2vec2-base-960h,rav
4,train,beer_Anger,0.000000,lj_finetune_facebook_wav2vec2-base-960h,rav


In [11]:
from IPython.display import Markdown

for split in metrics.split.unique():
    # global metrics
    global_only_metrics = metrics.loc[
        ~metrics.metric.str.contains("_") ^ metrics.metric.str.contains("total")
    ]
    global_only_metrics = global_only_metrics.assign(
        metric=global_only_metrics.metric.apply(lambda s: s.split("_")[0])
    )

    global_pivot_metrics = (
        global_only_metrics.query("split == @split").pivot(
            index="experiment", columns=["dataset", "metric"], values="value"
        )
        * 100
    )

    if split == "test":
        global_pivot_metrics.loc["lj_finetune_lit", ("rav", "eer")] = 18.01
        global_pivot_metrics.loc["lj_featureextract_lit", ("rav", "eer")] = 43.47
        global_pivot_metrics.loc["yuan_lit", ("iem", "beer")] = 100 - 75.6
        global_pivot_metrics.loc["yuan_lit", ("iem", "eer")] = 100 - 75.5

    # beer per emo
    per_emo_metrics = metrics.loc[
        metrics.metric.str.contains("_") & ~metrics.metric.str.contains("total")
    ]
    per_emo_metrics = per_emo_metrics.assign(
        emotion=per_emo_metrics.metric.apply(lambda s: s.split("_")[-1])
    )

    per_emo_pivot_metrics = (
        per_emo_metrics.query("split == @split").pivot(
            index="experiment", columns=["dataset", "emotion"], values="value"
        )
        * 100
    )

    # print
    display(Markdown(f"# {split}"))
    display(global_pivot_metrics.round(2).fillna("--"))
    display(per_emo_pivot_metrics.round(2).fillna("--"))

# train

dataset                                          rav           iem       
metric                                           eer   beer    eer   beer
experiment                                                               
lj_featureextract_facebook_wav2vec2-base-960h  33.68  33.68  37.72  40.26
lj_featureextract_facebook_wav2vec2-large      39.93  39.93  41.14  44.05
lj_finetune_facebook_wav2vec2-base-960h         5.90   5.90  36.49  39.99
lj_finetune_facebook_wav2vec2-large             0.81   0.81  29.16  29.77
yuan                                           84.14  84.14  74.10  74.57

dataset                                             rav                        \
emotion                                       Happiness  Anger Sadness   Fear   
experiment                                                                      
lj_featureextract_facebook_wav2vec2-base-960h     47.92  22.92   30.56  40.28   
lj_featureextract_facebook_wav2vec2-large         48.61  28.47   37.50  40.28   
lj_finetune_facebook_wav2vec2-base-960h           21.53   0.00    6.25   0.69   
lj_finetune_facebook_wav2vec2-large                0.69   0.00    2.08   0.69   
yuan                                              87.50  88.19   81.94  86.11   

dataset                                                              iem  \
emotion                                       Disgust Surprise Happiness   
experiment                                                                 
lj_featureextract_facebook_wav2vec2-base-960h   31.94    28.47     48.70   
lj_featureextract_facebook_wav2vec2-large       44.44    40.28     58.58   
lj_finetune_facebook_wav2vec2-base-960h          4.17     2.78     35.08   
lj_finetune_facebook_wav2vec2-large              0.69     0.69     63.87   
yuan                                            81.94    79.17     70.94   

dataset                                                               
emotion                                        Anger Sadness Neutral  
experiment                                                            
lj_featureextract_facebook_wav2vec2-base-960h  18.60   51.52   42.24  
lj_featureextract_facebook_wav2vec2-large      18.86   56.00   42.76  
lj_finetune_facebook_wav2vec2-base-960h        10.54   59.09   55.27  
lj_finetune_facebook_wav2vec2-large             6.35   17.88   30.97  
yuan                                           70.54   78.05   78.76

# val

dataset                                          rav           iem       
metric                                           eer   beer    eer   beer
experiment                                                               
lj_featureextract_facebook_wav2vec2-base-960h  43.75  43.75  43.75  44.87
lj_featureextract_facebook_wav2vec2-large      47.92  47.92  44.71  45.55
lj_finetune_facebook_wav2vec2-base-960h        31.25  31.25  44.23  49.33
lj_finetune_facebook_wav2vec2-large            16.67  16.67  39.42  41.01
yuan                                           81.25  81.25  76.92  76.22

dataset                                             rav                      \
emotion                                       Happiness Anger Sadness  Fear   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h      87.5  12.5    12.5  25.0   
lj_featureextract_facebook_wav2vec2-large          87.5  25.0     0.0  50.0   
lj_finetune_facebook_wav2vec2-base-960h            75.0  12.5    12.5  50.0   
lj_finetune_facebook_wav2vec2-large                 0.0  12.5     0.0  50.0   
yuan                                               87.5  87.5    75.0  87.5   

dataset                                                              iem  \
emotion                                       Disgust Surprise Happiness   
experiment                                                                 
lj_featureextract_facebook_wav2vec2-base-960h    37.5     87.5     58.46   
lj_featureextract_facebook_wav2vec2-large        37.5     87.5     57.38   
lj_finetune_facebook_wav2vec2-base-960h          12.5     25.0     50.98   
lj_finetune_facebook_wav2vec2-large              12.5     25.0     74.51   
yuan                                             75.0     75.0     70.00   

dataset                                                               
emotion                                        Anger Sadness Neutral  
experiment                                                            
lj_featureextract_facebook_wav2vec2-base-960h  22.58   55.88   42.55  
lj_featureextract_facebook_wav2vec2-large      28.33   58.97   37.50  
lj_finetune_facebook_wav2vec2-base-960h        20.78   64.71   60.87  
lj_finetune_facebook_wav2vec2-large            18.18   23.53   47.83  
yuan                                           78.12   74.29   82.46

# test

dataset                                          rav           iem       
metric                                           eer   beer    eer   beer
experiment                                                               
lj_featureextract_facebook_wav2vec2-base-960h   52.5   52.5  47.06  47.93
lj_featureextract_facebook_wav2vec2-large      52.92  52.92  48.63   48.4
lj_finetune_facebook_wav2vec2-base-960h        34.58  34.58  39.02  41.51
lj_finetune_facebook_wav2vec2-large             27.5   27.5  35.29  34.54
yuan                                           82.08  82.08  75.69  77.03
lj_finetune_lit                                18.01     --     --     --
lj_featureextract_lit                          43.47     --     --     --
yuan_lit                                          --     --   24.5   24.4

dataset                                             rav                      \
emotion                                       Happiness Anger Sadness  Fear   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h      55.0  37.5    32.5  70.0   
lj_featureextract_facebook_wav2vec2-large          65.0  40.0    25.0  65.0   
lj_finetune_facebook_wav2vec2-base-960h            67.5  15.0    30.0  37.5   
lj_finetune_facebook_wav2vec2-large                25.0  22.5    10.0  45.0   
yuan                                               80.0  92.5    85.0  85.0   

dataset                                                              iem  \
emotion                                       Disgust Surprise Happiness   
experiment                                                                 
lj_featureextract_facebook_wav2vec2-base-960h    65.0     55.0     54.32   
lj_featureextract_facebook_wav2vec2-large        62.5     60.0     63.58   
lj_finetune_facebook_wav2vec2-base-960h          35.0     22.5     37.04   
lj_finetune_facebook_wav2vec2-large              47.5     15.0     57.41   
yuan                                             72.5     77.5     68.52   

dataset                                                               
emotion                                        Anger Sadness Neutral  
experiment                                                            
lj_featureextract_facebook_wav2vec2-base-960h  21.58   64.29   51.55  
lj_featureextract_facebook_wav2vec2-large      22.30   63.39   44.33  
lj_finetune_facebook_wav2vec2-base-960h        10.07   64.29   54.64  
lj_finetune_facebook_wav2vec2-large            10.79   27.68   42.27  
yuan                                           72.66   80.36   86.60